In [ ]:
import ccxt
import psycopg2
import time
from google.colab import userdata

DB_URL = userdata.get('NEON_CONNECTION_STRING')

def run_smart_ingestion(iterations=120, delay=5):
    conn = None
    # 1. CHANGE THE EXCHANGE TO KRAKEN (US-Friendly)
    exchange = ccxt.kraken()

    try:
        print("Connecting to Neon...")
        conn = psycopg2.connect(DB_URL)
        conn.autocommit = True
        cur = conn.cursor()

        # 2. GET ASSET IDs
        print("Verifying Asset IDs...")
        cur.execute("SELECT ticker, asset_id FROM assets WHERE ticker IN ('BTC', 'ETH')")
        rows = cur.fetchall()
        display(rows)
        asset_map = {ticker: asset_id for ticker, asset_id in rows}
        print(f"Found Assets: {asset_map}")

        if 'BTC' not in asset_map or 'ETH' not in asset_map:
            print("ERROR: Assets missing! Run the INSERT INTO assets SQL in Neon first.")
            return

        # 3. START LOOP
        for i in range(iterations):
            # Fetch Prices (Using USD pairs for Kraken)
            btc_price = exchange.fetch_ticker('BTC/USD')['last']
            eth_price = exchange.fetch_ticker('ETH/USD')['last']

            # Insert BTC
            cur.execute(
                "INSERT INTO price_logs (asset_id, price) VALUES (%s, %s)",
                (asset_map['BTC'], btc_price)
            )

            # Insert ETH
            cur.execute(
                "INSERT INTO price_logs (asset_id, price) VALUES (%s, %s)",
                (asset_map['ETH'], eth_price)
            )

            print(f"[{i+1}/{iterations}] Saved - BTC: ${btc_price} | ETH: ${eth_price}")
            time.sleep(delay)

    except Exception as e:
        print(f"ERROR: {e}")
    finally:
        if conn:
            conn.close()
            print("Connection closed.")

run_smart_ingestion()

Connecting to Neon...
Verifying Asset IDs...


[('BTC', 1), ('ETH', 2)]

Found Assets: {'BTC': 1, 'ETH': 2}
[1/120] Saved - BTC: $87277.1 | ETH: $2930.78
[2/120] Saved - BTC: $87277.1 | ETH: $2930.78
[3/120] Saved - BTC: $87277.1 | ETH: $2931.29
[4/120] Saved - BTC: $87277.1 | ETH: $2931.29
[5/120] Saved - BTC: $87277.1 | ETH: $2931.55
[6/120] Saved - BTC: $87277.1 | ETH: $2931.55
[7/120] Saved - BTC: $87296.6 | ETH: $2931.55
[8/120] Saved - BTC: $87297.5 | ETH: $2932.0
[9/120] Saved - BTC: $87297.5 | ETH: $2932.0
[10/120] Saved - BTC: $87338.3 | ETH: $2932.32
[11/120] Saved - BTC: $87338.3 | ETH: $2933.08
[12/120] Saved - BTC: $87378.4 | ETH: $2935.36
[13/120] Saved - BTC: $87393.1 | ETH: $2935.5
[14/120] Saved - BTC: $87398.3 | ETH: $2935.37
[15/120] Saved - BTC: $87398.4 | ETH: $2934.92
[16/120] Saved - BTC: $87398.4 | ETH: $2934.94
[17/120] Saved - BTC: $87398.4 | ETH: $2934.74
[18/120] Saved - BTC: $87398.3 | ETH: $2934.74
[19/120] Saved - BTC: $87398.4 | ETH: $2934.74
[20/120] Saved - BTC: $87398.4 | ETH: $2934.74
[21/120] Saved - BTC: $87398.4 | ETH: